In [1]:
from enum import Enum

class timeUnit(Enum):
    days        = 'D'
    #weeks      = 'W'
    months      = 'M'
    quarters    = 'Q'
    years       = 'Y'
    
    @staticmethod
    def fromChar(c):
        
        if      c.upper() == 'D':
            return timeUnit.days
        #elif    c.upper() == 'W':
        #    return timeUnit.months
        elif    c.upper() == 'M':
            return timeUnit.months
        elif    c.upper() == 'Q':
            return timeUnit.quarters
        elif    c.upper() == 'Y':
            return timeUnit.years
        else:
            raise AttributeError

In [2]:
class businessDayConvention(Enum):
    '''
    Those conventions have been checked in Fingal
    but the list might not be exhaustive
    '''
    unadjusted        = 'Unadj'
    following         = 'Following'
    preceding         = 'Preceding'
    modifiedFollowing = 'Mod Foll'
    modifiedPreceding = 'Mod Prec'
        
    @staticmethod
    def fromChar(c):
        if c.upper() == 'Unadj':
            return businessDayConvention.unadjusted
        elif c.upper() == 'Following':
            return businessDayConvention.following
        elif c.upper() == 'Preceding':
            return businessDayConvention.preceding
        elif c.upper() == 'Mod Foll':
            return businessDayConvention.modifiedFollowing
        elif c.upper() == 'Mod Prec':
            return businessDayConvention.modifiedPreceding
        else:
            raise AttributeError

In [4]:
class period(object):
    '''
    classdocs
    '''
    
    # constructors
    def __init__(self, *args):
        
        if args is None or len(args) == 0:
            self.amount = 0
            self.unit = timeUnit.days
            
        elif len(args) == 1:
            self.amount, self.unit = self.__parse(args[0])
            
        elif len(args) == 2:
            self.amount = args[0]
            
            if type(args[1]) is timeUnit:
                self.unit   = args[1]
            elif type(args[1]) is str:
                self.unit = timeUnit.fromChar(args[1])
            else:
                raise AttributeError
        else:
            raise AttributeError

    def __repr__(self):
        
        if self.amount == float('-inf'):
            return '-inf'
        elif self.amount == float('inf'):
            return 'inf'
        else:
            return "{0}{1}".format(self.amount, self.unit.value)
    # overload the print function
    def __str__(self):
        return self.__repr__()
    
    # comparison operators
    def __eq__(self, other):
        return not(self < other or other < self)
    
    def __gt__(self, other):
        return other < self
    
    def __ge__(self, other):
        return not (self < other)
    
    def __le__(self, other):
        return not (self > other)
    
    class comparison(object):
        '''
        an internal class for period rough comparison
        '''
        def __init__(self, p_):
            if p_.unit == timeUnit.days:
                self.low  = p_.amount
                self.high = p_.amount
            #elif p_.unit == timeUnit.weeks:
            #    self.low = self.high = 7 * p_.amount
            elif p_.unit == timeUnit.months:
                self.low = 28 * p_.amount
                self.high = 31 * p_.amount
            elif p_.unit == timeUnit.quarters:
                self.low = 89 * p_.amount
                self.high = 91 * p_.amount
            elif p_.unit == timeUnit.years:
                self.low = 365 * p_.amount
                self.high = 366 * p_.amount
            else:
                raise AttributeError
            
    def __lt__(self, other):
        if (self.unit == other.unit):
            return self.amount < other.amount
        
        if (self.unit == timeUnit.quarters and other.unit == timeUnit.years):
            return self.amount < 4 * other.amount
        if (self.unit == timeUnit.years and other.unit == timeUnit.quarters):
            return 4 * self.amount < other.amount
        if (self.unit == timeUnit.months and other.unit == timeUnit.years):
            return self.amount < 12 * other.amount
        if (self.unit == timeUnit.years and other.unit == timeUnit.months):
            return 12 * self.amount < other.amount
        if (self.unit == timeUnit.months and other.unit == timeUnit.quarters):
            return self.amount < 3 * other.amount
        if (self.unit == timeUnit.quarters and other.unit == timeUnit.months):
            return 3 * self.amount < other.amount
        #if (self.unit == timeUnit.days and other.unit == timeUnit.weeks):
        #    return self.amount < 7 * other.amount
        #if (self.unit == timeUnit.weeks and other.unit == timeUnit.days):
        #    return 7 * self.amount < other.amount    
        
        # rough comparisons
        c1 = period.comparison(self); c2 = period.comparison(other)
        if c1.high < c2.low or c2.high < c1.low:
            return c1.high < c2.low
        else:
            raise AttributeError("period comparison cannot be performed between {0} and {1}".format(self, other))
        
    # parse from string
    @staticmethod
    def __parse(args):
        return int(args[:-1]), timeUnit.fromChar(args[-1])

In [8]:
class bucket(object):
    '''
    a class representing a time bucket.
    - The time bucket may be single sided
      and the missing bucket will be assumed unended
    - Each end can be included or not.
    - Arithmetic operations must include valid unions, intersections
    - Also, we test for the inclusion of a period in the bucket
    - it provides comparison operator <, >, ... 
      only correctly defined for non overlapping cases
    '''
    class edge(object):
        '''
        a class representing a edge of an interval.
        a simple tuple of (period, bool) with comparison operators
        empty edges represents +/- infty
        '''
        def __init__(self, period_ = None, include_ = None):    
            if period_ is not None and include_ is None:
                self.include = False
            else:
                self.include = include_
            self.period = period_
            
        # comparison operators
        def __eq__(self, other):
            return (self.period == other.period and self.include == other.include)
        
    def __init__(self, lhs_, rhs_, includeLhs_, includeRhs_):
        
        '''
        Constructor
        - if lhs_ > rhs_, the two variables will be swapped
        - the inclusions default to false
        - sigletons (lhs_ = rhs_) are also supported 
          but then the inclusion is forced to true
        '''
        # we need to preserve order
        if lhs_ is not None and rhs_ is not None and rhs_ < lhs_:
            rhs_, lhs_ = lhs_, rhs_            # swap
            includeLhs_, includeRhs_ = includeRhs_, includeLhs_
        
        if lhs_ is not None and rhs_ is not None and lhs_ == rhs_:
            self.lhs = bucket.edge(lhs_, True)
            self.rhs = bucket.edge(rhs_, True)
            return 
        
        if lhs_ is None:
            lhs_ = period(float('-inf'), timeUnit.years)
        
        if rhs_ is None:
            rhs_ = period(float('inf'), timeUnit.years)
            
        if (includeLhs_ is not None):
            self.lhs = bucket.edge(lhs_, includeLhs_)
        else:
            self.lhs = bucket.edge(lhs_, False)
        
        if (includeRhs_ is not None):
            self.rhs = bucket.edge(rhs_, includeRhs_)
        else:
            self.rhs = bucket.edge(rhs_, False)
        return
    
    def __repr__(self):
        
        retVal = None
        if self.lhs.include:
            retVal = '['
        else:
            retVal = ']'
    
        retVal = retVal + str(self.lhs.period) + ':' + str(self.rhs.period)
            
        if self.rhs.include:
            retVal = retVal + ']'
        else:
            retVal = retVal + '['
            
        return retVal
    
    # overload the print function
    
    def __str__(self):
        return self.__repr__()
    
    def __add__(self, other):
        if self.rhs.period < other.lhs.period or other.rhs.period < self.lhs.period:
            raise AttributeError("intervals {0} and {1} are not suitable for addition".format(self, other))
        elif self.rhs.period == other.lhs.period and not (self.rhs.include or other.lhs.include):
            raise AttributeError("intervals {0} and {1} are not suitable for addition".format(self, other))
        elif other.rhs.period == self.lhs.period and not (other.rhs.include or self.lhs.include):
            raise AttributeError("intervals {0} and {1} are not suitable for addition".format(self, other))
        else:
            lVal = None; rVal = None
            if self.lhs.period < other.lhs.period:
                lVal = self.lhs
            elif other.lhs.period < self.lhs.period:
                lVal = other.lhs
            else:
                lVal = bucket.edge(self.lhs.period, self.lhs.include or other.lhs.include)
                
            if self.rhs.period < other.rhs.period:
                rVal = other.rhs
            elif other.rhs.period < self.rhs.period:
                rVal = self.rhs
            else:
                rVal = bucket.edge(self.rhs.period, self.rhs.include or other.rhs.include)
                
            return bucket(lVal.period, rVal.period, lVal.include, rVal.include)    

    # comparison operators
    def __eq__(self, other):
        return (self.lhs == other.lhs and self.rhs == other.rhs)
    
    def __gt__(self, other):
        return other < self
    
    def __ge__(self, other):
        return not (self < other)
    
    def __le__(self, other):
        return not (self > other)
    
    def __lt__(self, other):
        if self.rhs.period < other.lhs.period:
            return True
        elif self.rhs.period == other.lhs.period and not self.rhs.include * other.lhs.include:
            return True
        else:
            return False
    
    def contains(self, period_):
        if period_ > self.rhs.period:
            return False 
        elif period_ < self.lhs.period:
            return False
        elif period_ == self.rhs.period and not self.rhs.include:
            return False
        elif period_ == self.lhs.period and not self.lhs.include:
            return False
        else:
            return True
        
    @staticmethod
    def fromString(args):
        inclLhs = None; inclRhs = None
        lhs = None; rhs = None
        if args[0] == "[":
            inclLhs = True
        elif args[0] == "]":
            inclLhs = False
        else:
            raise AttributeError("{0} is not a valid bucket pattern".format(args))
        
        if args[-1] == "[":
            inclRhs = False
        elif args[-1] == "]":
            inclRhs = True
        else:
            raise AttributeError("{0} is not a valid bucket pattern".format(args))
        
        p = args[1:-1].split(':')
        
        if len(p) != 2:
            raise AttributeError("{0} is not a valid bucket pattern".format(args))
        else:
            if p[0] != '-inf':
                lhs = period(p[0])
            if p[1] != 'inf':
                rhs = period(p[1])
            return bucket(lhs, rhs, inclLhs, inclRhs)  

In [10]:
import six
from abc import ABCMeta, abstractmethod

@six.add_metaclass(ABCMeta)
class abstractFactory(object):
    '''
    an abstract factory class
    '''
    def __init__(self, params):
        '''
        Constructor
        '''
        
    @abstractmethod
    def createInstance(self, node):
        pass

In [11]:
import six

from enum import Enum
from abc import ABCMeta, abstractmethod

class level(Enum):
    high        = 3
    medium      = 2
    low         = 1
        
    def __eq__(self, other):
        return self.value == other.value
    
    def __gt__(self, other):
        return self.value > other.value
    
    def __lt__(self, other):
        return self.value < other.value
    
    def __ge__(self, other):
        return not (self < other)
    
    def __le__(self, other):
        return not (self > other)
    
    def __str__(self):
        if self == level.high:
            return 'high'
        elif self == level.medium:
            return 'medium'
        elif self == level.low:
            return 'low'
        else:
            raise AttributeError
    
    @staticmethod
    def fromChar(c):
        if c.upper() == 'HIGH':
            return level.high
        elif c.upper() == 'MEDIUM':
            return level.medium
        elif c.upper() == 'LOW':
            return level.low
        else:
            raise AttributeError
        
class logType(Enum):
    info    = 'info'
    warning = 'warning'
    error   = 'error'
    debug   = 'debug'
    
    @staticmethod
    def fromChar(c):
        if c.upper() == 'INFO':
            return logType.info
        elif c.upper() == 'WARNING':
            return logType.warning
        elif c.upper() == 'ERROR':
            return logType.error
        elif c.upper() == 'DEBUG':
            return logType.debug
        else:
            raise AttributeError
        
    def __str__(self):
        if self == logType.info:
            return 'info'
        elif self == logType.warning:
            return 'warning'
        elif self == logType.error:
            return 'error'
        elif self == logType.debug:
            return 'debug'
        else:
            raise AttributeError
        
@six.add_metaclass(ABCMeta)
class logger():
    '''
    classdocs
    '''      
    def __init__(self, threshold_):
        '''
        Constructor
        '''
        self.threshold_ = threshold_
        
    def add(self, message_, type_, verbosity_ = level.high, eventId_ = 0):
        if verbosity_ >= self.threshold_:
            self.addImpl(message_, type_, eventId_)
        
    @abstractmethod
    def addImpl(self, message_, type_, eventId_):
        pass

In [12]:
import threading

from multiprocessing import Lock

from datetime import datetime

class consoleLogger(logger):
    '''
    a console logger
    add method is thread safe
    '''

    def __init__(self, threshold = level.low):
        '''
        Constructor
        '''
        super(consoleLogger, self).__init__(threshold)
        
        self.mutex = Lock()
        
    def addImpl(self, message_, type_, eventId_):
        with self.mutex:
            print('[{0} : {1} : {2} : {3}] {4}'.format(
                datetime.now().isoformat(), 
                threading.current_thread().name, 
                eventId_, type_, message_))